# 🏀 NBA 2024-2025: Slumps & Recoveries
## Notebook 01 - Data Cleaning & Preparation, Part 1
This notebook loads, inspects, and cleans the qualified players' game logs from the 2024-2025 NBA regular season.

### Qualified Players
This analysis will only include NBA players **who made at least 300 field goals** during the 2024-2025 NBA regular season. This is Basketball Reference's qualification standard for Field Goal Percentage (FG%).

This cutoff ensures each player **had a large enough shooting sample** for their slump/recovery patterns to be meaningful.

### Where Do The Game Logs Come From?
For example, here are the 2024-2025 game logs for Luka Dončić (my favorite NBA player):

[Luka Dončić's 2024-2025 game logs](https://www.basketball-reference.com/players/d/doncilu01/gamelog/2025#header)

The game logs for all **117 qualified players** were manually copied and pasted (one by one) into one large `.txt` file, which will be uploaded below.

In [4]:
# Import libraries
import pandas as pd
import numpy as np
import re

---
## Load Raw Data

In [6]:
# Load raw NBA 2024-2025 qualified players' game logs
df = pd.read_csv(r"C:\Users\dylan\OneDrive\Documents\Portfolio_Projects\project10_NBA_2025_game_logs\01_original_data\to_clean\NBA_game_logs_2024_25.txt")
df

,Player,Gcar,Gtm,Date,Team,Unnamed: 5,Opp,Result,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,Jarrett Allen,Gcar,Gtm,Date,Team,NaN,Opp,Result,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
1,1,487,1,2024-10-23,CLE,@,TOR,W 136-106,*,26:05,...,5,7,1,1,4,1,1,14,16.7,8
2,2,488,2,2024-10-25,CLE,NaN,DET,W 113-101,*,31:13,...,11,11,2,1,1,1,0,17,19.1,3
3,3,489,3,2024-10-26,CLE,@,WAS,W 135-116,*,25:04,...,4,5,1,1,2,0,1,23,24.2,13
4,4,490,4,2024-10-28,CLE,@,NYK,W 110-104,*,31:24,...,10,15,1,1,0,2,2,15,16.7,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9715,76,158,78,2025-04-06,BRK,NaN,TOR,L 109-120,*,20:26,...,7,7,4,0,1,3,0,13,10.5,2
9716,76,NaN,79,2025-04-08,BRK,NaN,NOP,W 119-114,Inactive,Inactive,...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive
9717,77,159,80,2025-04-10,BRK,NaN,ATL,L 109-133,*,17:13,...,3,3,3,0,0,6,4,10,-0.1,-17
9718,78,160,81,2025-04-11,BRK,@,MIN,L 91-117,*,23:53,...,4,5,6,1,0,1,2,20,22.4,-7


___
## Initial Checks

In [8]:
# Rows and columns
print(f"Number of rows: {df.shape[0]:,}")
print(f"Number of columns: {df.shape[1]:,}")

Number of rows: 9,720
Number of columns: 34


In [9]:
# All column names
df.columns

Index(['Player', 'Gcar', 'Gtm', 'Date', 'Team', 'Unnamed: 5', 'Opp', 'Result',
       'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%',
       'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS', 'GmSc', '+/-'],
      dtype='object')

___
## Clean

In [11]:
# Drop full season statistics rows (i.e., where Date is NaN)
df = df.dropna(subset=["Date"]).reset_index(drop=True)

In [12]:
# Detect where new player block starts
mask = df["Gcar"] == "Gcar"

# Create new `player_name` column
df.loc[mask, "player_name"] = df.loc[mask, "Player"]

# Forward fill player names down each player's block
df["player_name"] = df["player_name"].ffill()

In [13]:
# Drop all internal header rows (e.g., where df["Gcar"] = "Gcar")
df = df[df["Gcar"] != "Gcar"].reset_index(drop=True)

In [14]:
# Reorder columns so that `player_name` column is first
cols = ["player_name"] + [c for c in df.columns if c != "player_name"]
df = df[cols]

In [15]:
# Drop unnecessary columns
df = df.drop(columns=["Player", "Gcar", "GmSc"], errors="ignore")

In [16]:
# Rename columns for clarity
df = df.rename(columns={
    "Gtm": "team_game",
    "Date": "game_date",
    "Unnamed: 5": "home_or_away",
    "Opp": "opponent",
    "GS": "started_game",
    "MP": "minutes_played",
    "FG": "field_goals_made",
    "FGA": "field_goals_attempted",
    "FG%": "field_goal_pct",
    "3P": "three_pointers_made",
    "3PA": "three_pointers_attempted",
    "3P%": "three_point_pct",
    "2P": "two_pointers_made",
    "2PA": "two_pointers_attempted",
    "2P%": "two_point_pct",
    "eFG%": "effective_fg_pct",
    "FT": "free_throws_made",
    "FTA": "free_throws_attempted",
    "FT%": "free_throw_pct",
    "ORB": "offensive_rebounds",
    "DRB": "defensive_rebounds",
    "TRB": "total_rebounds",
    "AST": "assists",
    "STL": "steals",
    "BLK": "blocks",
    "TOV": "turnovers",
    "PF": "personal_fouls",
    "PTS": "points",
    "+/-": "plus_minus"
})

# Ensure all column names are lowercase
df.columns = df.columns.str.lower()

In [17]:
# Set up `home_or_away` values
df["home_or_away"] = df["home_or_away"].replace({"@": "away"}).fillna("home")

In [18]:
# Set up `started_game` values
df["started_game"] = (
    df["started_game"]
    .replace({
        "*": True,
        "Inactive": False,
        "Did Not Dress": False,
        "Did Not Play": False,
        "Not With Team": False,
        "Suspended": False,
        np.nan: False
    }).astype(bool)
)

C:\Users\dylan\AppData\Local\Temp\ipykernel_27636\3207929300.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({


In [19]:
# Extract W/L from `result`
df["wl"] = df["result"].str[0]

# Extract score from `result` (e.g. "115-110")
df["score"] = df["result"].str.extract(r"(\d+-\d+)")

# Extract overtime indicator (did game go to OT?)
df["overtime"] = df["result"].str.contains(r"\(.*OT\)", regex=True)

In [20]:
# Extract team and opponent point totals from `score`
df[["team_points", "opponent_points"]] = df["score"].str.split("-", expand=True)
df[["team_points", "opponent_points"]] = df[["team_points", "opponent_points"]].apply(pd.to_numeric, errors="coerce").astype("Int64")

# Compute point differential
df["point_diff"] = df["team_points"] - df["opponent_points"]

# Drop `result` and `score` columns (no longer necessary)
df = df.drop(columns=["result", "score"], errors="coerce")

In [21]:
# Create a `shoots` column (right-handed or left-handed)
df["shoots"] = np.select(
    [df["player_name"].str.contains(r"\*", regex=True)],
    ["Left"],
    default="Right"
)

# Remove any * from `player_name` column
df["player_name"] = df["player_name"].str.replace(r"\*", "", regex=True).str.strip()

In [22]:
# Strip any stray spaces in all object/string columns
for col in df.select_dtypes(include=["object", "string"]).columns:
    df[col] = df[col].str.strip()

___
## Data Types

In [24]:
# Check data types
df.dtypes

player_name                 object
team_game                   object
game_date                   object
team                        object
home_or_away                object
opponent                    object
started_game                  bool
minutes_played              object
field_goals_made            object
field_goals_attempted       object
field_goal_pct              object
three_pointers_made         object
three_pointers_attempted    object
three_point_pct             object
two_pointers_made           object
two_pointers_attempted      object
two_point_pct               object
effective_fg_pct            object
free_throws_made            object
free_throws_attempted       object
free_throw_pct              object
offensive_rebounds          object
defensive_rebounds          object
total_rebounds              object
assists                     object
steals                      object
blocks                      object
turnovers                   object
personal_fouls      

In [25]:
# Convert `minutes_played` from mm:ss to float
def convert_minutes_to_float(x):
    if pd.isna(x) or x == "" or ":" not in str(x):
        return np.nan
    try:
        m, s = str(x).split(":")
        return int(m) + int(s)/60
    except ValueError:
        return np.nan

df["minutes_played"] = df["minutes_played"].apply(convert_minutes_to_float)

In [26]:
# Define column groups by data type
date_cols = ["game_date"]

int_cols = [
    "team_game", "team_points", "opponent_points", "point_diff",
    "field_goals_made", "field_goals_attempted",
    "three_pointers_made", "three_pointers_attempted",
    "two_pointers_made", "two_pointers_attempted",
    "free_throws_made", "free_throws_attempted",
    "offensive_rebounds", "defensive_rebounds", "total_rebounds",
    "assists", "steals", "blocks", "turnovers", "personal_fouls",
    "points", "plus_minus"
]

float_cols = [
    "minutes_played", "field_goal_pct", "three_point_pct",
    "two_point_pct", "effective_fg_pct", "free_throw_pct"
]

In [27]:
# Apply data type conversions
df[date_cols] = df[date_cols].apply(pd.to_datetime, errors="coerce")

df[int_cols] = df[int_cols].apply(pd.to_numeric, errors="coerce")
df[int_cols] = df[int_cols].round(0)
df[int_cols] = df[int_cols].astype("Int64")

df[float_cols] = df[float_cols].apply(pd.to_numeric, errors="coerce").astype(float)

In [28]:
# Confirm updated data types
df.dtypes

player_name                         object
team_game                            Int64
game_date                   datetime64[ns]
team                                object
home_or_away                        object
opponent                            object
started_game                          bool
minutes_played                     float64
field_goals_made                     Int64
field_goals_attempted                Int64
field_goal_pct                     float64
three_pointers_made                  Int64
three_pointers_attempted             Int64
three_point_pct                    float64
two_pointers_made                    Int64
two_pointers_attempted               Int64
two_point_pct                      float64
effective_fg_pct                   float64
free_throws_made                     Int64
free_throws_attempted                Int64
free_throw_pct                     float64
offensive_rebounds                   Int64
defensive_rebounds                   Int64
total_rebou

___
## Date Range

In [30]:
# Date range of all regular season games
date_min = df["game_date"].min().date()
date_max = df["game_date"].max().date()
print(f"Date range of all games in 2024-2025 NBA regular season: \n{date_min} - {date_max}")

Date range of all games in 2024-2025 NBA regular season: 
2024-10-22 - 2025-04-13


___
## Final Sanity Checks

In [32]:
# Confirm that there are 117 player names
print(f"Unique player names: {df["player_name"].nunique():,}")

Unique player names: 117


In [33]:
# Post-cleaning rows and columns
print(f"Number of rows: {df.shape[0]:,}")
print(f"Number of columns: {df.shape[1]:,}")

Number of rows: 9,603
Number of columns: 37


In [34]:
# Loop through all columns to confirm their unique values (i.e., data auditing to search for any anomalies)
for col in df.columns:
    uniques = df[col].dropna().unique()
    print(f"\n{col}:")
    print(uniques)


player_name:
['Jarrett Allen' 'Jalen Duren' 'Rudy Gobert' 'Ivica Zubac' 'Jakob Poeltl'
 'Giannis Antetokounmpo' 'Domantas Sabonis' 'Christian Braun'
 'Nikola Jokić' 'Onyeka Okongwu' 'Nic Claxton' 'Evan Mobley'
 'Amen Thompson' 'Jonas Valančiūnas' 'Nikola Vučević' 'Obi Toppin'
 'Kevin Durant' 'Derrick Jones Jr.' 'Karl-Anthony Towns' 'Josh Hart'
 'Shai Gilgeous-Alexander' 'T.J. McConnell' 'Pascal Siakam'
 'Anthony Davis' 'Ty Jerome' 'LeBron James' 'Zach LaVine' 'Brook Lopez'
 'Naji Marshall' 'Harrison Barnes' 'Jimmy Butler' 'Michael Porter Jr.'
 'Mikal Bridges' 'Kawhi Leonard' 'Bradley Beal' 'Alperen Şengün'
 'Dyson Daniels' 'Jaren Jackson Jr.' 'Aaron Wiggins' 'Jalen Brunson'
 'Bam Adebayo' 'Julius Randle' 'Desmond Bane' 'Jalen Williams'
 'Norman Powell' 'Santi Aldama' 'Keldon Johnson' 'Myles Turner'
 'Collin Sexton' 'Jaden McDaniels' 'DeMar DeRozan' 'Tobias Harris'
 'Deni Avdija' 'OG Anunoby' 'Victor Wembanyama' 'Cameron Johnson'
 'Jamal Murray' 'Tyrese Haliburton' 'Kyrie Irving' 'Tyle

## 🔍 Percentage Validity Check
`three_point_pct`, `free_throw_pct`, and `effective_fg_pct` contain **integer values greater than 1** (e.g., 2, 5, 7). Since traditional shooting percentages must fall between **0 and 1**, these values will be investigated further.

For `effective_fg_pct`, any values *slightly* above 1.0 (e.g., 1.125) are acceptable because eFG% provides extra weighting (+0.5) for 3PT makes. A player who makes a ton of 3-point attempts relative to their total field goal attempts can end up having an eFG% slightly over 1.0 for that game.

In [36]:
# Locate `three_point_pct` values > 1.0
df.loc[df["three_point_pct"] > 1, ["player_name", "game_date", "three_pointers_made", "three_pointers_attempted", "three_point_pct"]]

,player_name,game_date,three_pointers_made,three_pointers_attempted,three_point_pct
82,Jalen Duren,2024-10-23,0,0,4.0
83,Jalen Duren,2024-10-25,0,0,2.0
84,Jalen Duren,2024-10-26,0,0,7.0
85,Jalen Duren,2024-10-28,0,0,5.0
87,Jalen Duren,2024-11-01,0,0,2.0
...,...,...,...,...,...
323,Ivica Zubac,2025-04-05,0,0,11.0
324,Ivica Zubac,2025-04-08,0,0,11.0
325,Ivica Zubac,2025-04-09,0,0,9.0
326,Ivica Zubac,2025-04-11,0,0,8.0


## 🔍
All `three_point_pct` values **greater than 1.0** come from players who recorded **0 makes and 0 attempts** from three (in these games). With 0 makes and 0 attempts, the `three_point_pct` for these games should be `NaN`, but for some reason, these integers appear.

In [38]:
# Locate `free_throw_pct` values > 1.0
df.loc[df["free_throw_pct"] > 1, ["player_name", "game_date", "free_throws_made", "free_throws_attempted", "free_throw_pct"]]

,player_name,game_date,free_throws_made,free_throws_attempted,free_throw_pct
82,Jalen Duren,2024-10-23,6,1,4.0
83,Jalen Duren,2024-10-25,4,0,3.0
84,Jalen Duren,2024-10-26,2,0,3.0
85,Jalen Duren,2024-10-28,0,<NA>,4.0
86,Jalen Duren,2024-10-30,2,0,3.0
...,...,...,...,...,...
322,Ivica Zubac,2025-04-04,4,0,7.0
323,Ivica Zubac,2025-04-05,3,1,2.0
324,Ivica Zubac,2025-04-08,2,1,8.0
326,Ivica Zubac,2025-04-11,1,1,2.0


## 🔍
All `free_throw_pct` values **greater than 1.0** come from players who, in most cases, had more `free_throws_made` than `free_throws_attempted`. This is obviously impossible.

In [40]:
# Find all players without a 3 point attempt for entire season
zero_3pt_players = df.groupby("player_name")["three_pointers_attempted"].sum()
zero_3pt_players = zero_3pt_players[zero_3pt_players == 0].index.tolist()
zero_3pt_players

['Ivica Zubac', 'Jalen Duren', 'Rudy Gobert']

## 🔍
The basketball-reference game logs for the **3 players who did not attempt a 3-point shot** during the 2024-2025 NBA regular season were further inspected:

- [Ivica Zubac's 2024-2025 game logs](https://www.basketball-reference.com/players/z/zubaciv01/gamelog/2025#header)
- [Jalen Duren's 2024-2025 game logs](https://www.basketball-reference.com/players/d/durenja01/gamelog/2025)
- [Rudy Gobert's 2024-2025 game logs](https://www.basketball-reference.com/players/g/goberru01/gamelog/2025#header)

It turns out that **there is no `three_point_pct` column** for any of these 3 players. Therefore, **all columns to the right of `three_point_pct` were shifted one column to the left** during the import process.

In [42]:
# View columns
df.loc[df["player_name"].isin(zero_3pt_players),
       ["player_name","game_date", "three_pointers_made","three_pointers_attempted",
        "three_point_pct","two_pointers_made","two_pointers_attempted", "two_point_pct",
        "effective_fg_pct"]]

,player_name,game_date,three_pointers_made,three_pointers_attempted,three_point_pct,two_pointers_made,two_pointers_attempted,two_point_pct,effective_fg_pct
82,Jalen Duren,2024-10-23,0,0,4.0,4,1,1.000,5.0
83,Jalen Duren,2024-10-25,0,0,2.0,4,0,0.500,1.0
84,Jalen Duren,2024-10-26,0,0,7.0,7,1,1.000,1.0
85,Jalen Duren,2024-10-28,0,0,5.0,6,1,0.833,0.0
86,Jalen Duren,2024-10-30,0,0,0.0,0,<NA>,NaN,0.0
...,...,...,...,...,...,...,...,...,...
323,Ivica Zubac,2025-04-05,0,0,11.0,11,1,1.000,3.0
324,Ivica Zubac,2025-04-08,0,0,11.0,17,1,0.647,2.0
325,Ivica Zubac,2025-04-09,0,0,9.0,13,1,0.692,2.0
326,Ivica Zubac,2025-04-11,0,0,8.0,10,1,0.800,1.0


All values for all columns to the right of `three_pointers_attempted` are incorrect here.

After several unsuccessful attempts at column re-shifting via code, it was decided that it would be best to go back to the raw `.txt` file and manually add the `three_point_pct` column for these 3 players with `NaN` as values. This will ensure that all columns are properly aligned with correct values.